В домашней работе необходимо с помощью FastAPI реализовать REST API:

На 3 балла. Твой REST API - это список покупок и содержит поля: название товара, группа товара (например, электроника или продовольствие), цена, единица измерения, количество. Также необходимо реализовать метод, который возвращает список - расходы по каждой группе товаров и сумму всех покупок. Также необходимо с помощью библиотеки requests продемонстрировать запросами к REST API, как работает ваш веб-сервис. Это задание можно сравнить с "покрытием тестами" вашего API. Нечто похожее делают тестировщики в ИТ-компаниях. Вам необходимо покрыть запросами все методы, которые вы реализуете на веб-сервере.

На 4 балла. Вам необходимо сделать красивую документацию для твоего REST API с подробным описанием. Для этого тебе придется обратиться к документации:

https://fastapi.tiangolo.com/ru/tutorial/metadata/

https://fastapi.tiangolo.com/ru/tutorial/path-operation-configuration/#response-description

На 5 баллов. Творческое задание. REST API можно использовать для взаимодействия с вашей моделью нейронной сети. Вы уже знаете, что можно обучить модель, а лучший результат выгрузить для дальнейшего использования. Для получения 5 баллов необходимо обучить свою модель, загрузить ее в Colab. Задача может быть любой: регрессии, классификации, входными данными могут быть картинки или текстовые данные. С помощью REST API обеспечьте взаимодействие с моделью. Это полностью творческое задание!

In [ ]:
!pip install fastapi # Установим FastAPI
!pip install uvicorn # Установим ASGI-сервер
!pip install python-multipart # Необходимая зависимость для FastAPI (для работы с данными отправленных форм на сайте)

In [ ]:
%%writefile main.py

import uuid
from fastapi import FastAPI, Body
from fastapi.responses import JSONResponse
from collections import defaultdict

# Определяем класс Purchase с полями:
class Purchase:
    def __init__(self, name, group, price, unit, quantity):
        self.name = name
        self.group = group
        self.price = price
        self.unit = unit
        self.quantity = quantity
        self.id = str(uuid.uuid4())


# условная база данных - набор объектов Purchase
purchases = [Purchase("Помидоры", "Продовольствие", 200, "кг", 2),
             Purchase("Телевизор", "Электроника", 18000, "шт", 2),
             Purchase("Миф", "Бытовая химия", 80, "шт", 5),
             Purchase("Сыр", "Продовольствие", 800, "кг", 0.5),
             Purchase("Арбуз", "Продовольствие", 40, "кг", 5),
             Purchase("Бумага А4", "Канцелярия", 300, "шт", 2),
             ]


# для поиска товара в списке products
def find_purchase(id):
    for purchase in purchases:
        if purchase.id == id:
            return purchase
    return None


app = FastAPI()


@app.get("/api/purchases",
         summary="Список покупок",
         description="Получение списка покупок из базы"
         )
async def get_purchases():
    return purchases


@app.get("/api/purchases/{id}",
         summary="Покупка",
         description="Получение Покупки по идентификатору")
async def get_purchase(id):
    # получаем покупку по id
    purchase = find_purchase(id)
    print(purchase)
    # если не найден, отправляем статусный код и сообщение об ошибке
    if not purchase:
        return JSONResponse(
            status_code=404,
            content={"message": "Покупка не найден"}
        )
    # если пользователь найден, отправляем его
    return purchase


# Получение суммы всех покупок и расходов по каждой группе
@app.get("/api/purchases_sum",
         summary="Сумма всех покупок",
         description="Получение суммы всех покупок и расходов по каждой группе")
async def get_purchases_sum():
    d = defaultdict(list)
    for purchase in purchases:
        d[purchase.group].append(purchase.price * purchase.quantity)
    purchases_group_sum = [(i, sum(k)) for i, k in d.items()]

    return purchases_group_sum, sum(i[1] for i in purchases_group_sum)


@app.post("/api/purchases",
          summary="Создание покупки",
          description="Добавление новой покупки в базу данных")
async def create_purchase(data=Body()):
    purchase = Purchase(data["name"], data["group"], data["price"], data["unit"], data["quantity"])
    # добавляем объект в список purchases
    purchases.append(purchase)
    return purchase


@app.put("/api/purchases",
         summary="Редактирование покупки",
         description="Редактирование покупки в базе данных")
async def edit_purchase(data=Body()):
    # получаем покупку по id
    purchase = find_purchase(data["id"])
    # если не найден, отправляем статусный код и сообщение об ошибке
    if not purchase:
        return JSONResponse(
            status_code=404,
            content={"message": "Покупка не найден"}
        )
    # если покупка найдена, изменяем её данные и отправляем обратно клиенту
    purchase.name = data["name"]
    purchase.group = data["group"]
    purchase.price = data["price"]
    purchase.unit = data["unit"]
    purchase.quantity = data["quantity"]
    return purchase


@app.delete("/api/purchases/{id}",
            summary="Удаление покупки",
            description="Удаление покупки из базы данных")
async def delete_purchase(id):
    # получаем покупку по id
    purchase = find_purchase(id)
    # если не найден, отправляем статусный код и сообщение об ошибке
    if not purchase:
        return JSONResponse(
            status_code=404,
            content={"message": "Покупка не найден"}
        )
    purchases.remove(purchase)
    return purchase

Overwriting main.py


In [ ]:
!nohup uvicorn main:app --reload &

nohup: appending output to 'nohup.out'


In [ ]:
import requests

# Получим список покупок
def get_purchases():
    print('\nСписок покупок:')
    r = requests.get('http://127.0.0.1:8000/api/purchases')
    purchases = r.json()
    for purchase in purchases:
        print(
            f"Покупка: {purchase['quantity']}{purchase['unit']} - {purchase['name']} ({purchase['group']}), цена {purchase['price']}р. за 1{purchase['unit']}, (id={purchase['id']})")

In [ ]:
get_purchases()


Список покупок:
Покупка: 2кг - Помидоры (Продовольствие), цена 200р. за 1кг, (id=2ec65fa9-72bf-484e-8c05-a088c84fdf33)
Покупка: 2шт - Телевизор (Электроника), цена 18000р. за 1шт, (id=d426ce87-510e-4a3b-900c-f7dba8b63d61)
Покупка: 5шт - Миф (Бытовая химия), цена 80р. за 1шт, (id=ccbfe4af-61f8-4fb3-bf92-9f6f3fb64213)
Покупка: 0.5кг - Сыр (Продовольствие), цена 800р. за 1кг, (id=530af903-d932-4498-bb8d-9bf892bc8b49)
Покупка: 5кг - Арбуз (Продовольствие), цена 40р. за 1кг, (id=f4b15204-86dc-409a-8a55-f0d09bae777f)
Покупка: 2шт - Бумага А4 (Канцелярия), цена 300р. за 1шт, (id=63f1606a-cbaf-4980-b047-135ac9137762)


In [ ]:
r = requests.get('http://127.0.0.1:8000/api/purchases')
purchases = r.json()

# Меняем вторую покупку
purchase = purchases[1]
purchase['name'] = "Смартфон"
purchase['group'] = "Электроника"
purchase['price'] = 12000
purchase['unit'] = 'шт'
purchase['quantity'] = 1
r = requests.put('http://127.0.0.1:8000/api/purchases', json=purchase)
purchase = r.json()
print(f"У покупки (id={purchase['id']}) новые значения: {purchase['quantity']}{purchase['unit']} - {purchase['name']} ({purchase['group']}), цена {purchase['price']}р. за 1{purchase['unit']}")

У покупки (id=d426ce87-510e-4a3b-900c-f7dba8b63d61) новые значения: 1шт - Смартфон (Электроника), цена 12000р. за 1шт


In [ ]:
get_purchases()


Список покупок:
Покупка: 2кг - Помидоры (Продовольствие), цена 200р. за 1кг, (id=2ec65fa9-72bf-484e-8c05-a088c84fdf33)
Покупка: 1шт - Смартфон (Электроника), цена 12000р. за 1шт, (id=d426ce87-510e-4a3b-900c-f7dba8b63d61)
Покупка: 5шт - Миф (Бытовая химия), цена 80р. за 1шт, (id=ccbfe4af-61f8-4fb3-bf92-9f6f3fb64213)
Покупка: 0.5кг - Сыр (Продовольствие), цена 800р. за 1кг, (id=530af903-d932-4498-bb8d-9bf892bc8b49)
Покупка: 5кг - Арбуз (Продовольствие), цена 40р. за 1кг, (id=f4b15204-86dc-409a-8a55-f0d09bae777f)
Покупка: 2шт - Бумага А4 (Канцелярия), цена 300р. за 1шт, (id=63f1606a-cbaf-4980-b047-135ac9137762)


In [ ]:
# Добавление новой покупки
new_purchase = {'name': 'Мыло', 'group': 'Бытовая химия', 'price': 70, 'unit': 'шт', 'quantity': 10}
r = requests.post('http://127.0.0.1:8000/api/purchases', json=new_purchase)
new_purchase = r.json()
print(f"Добавлена покупка: {new_purchase['quantity']}{new_purchase['unit']} {new_purchase['name']} ({new_purchase['group']}), цена {new_purchase['price']}р. за 1{new_purchase['unit']}")

Добавлена покупка: 10шт Мыло (Бытовая химия), цена 70р. за 1шт


In [ ]:
get_purchases()


Список покупок:
Покупка: 2кг - Помидоры (Продовольствие), цена 200р. за 1кг, (id=2ec65fa9-72bf-484e-8c05-a088c84fdf33)
Покупка: 1шт - Смартфон (Электроника), цена 12000р. за 1шт, (id=d426ce87-510e-4a3b-900c-f7dba8b63d61)
Покупка: 5шт - Миф (Бытовая химия), цена 80р. за 1шт, (id=ccbfe4af-61f8-4fb3-bf92-9f6f3fb64213)
Покупка: 0.5кг - Сыр (Продовольствие), цена 800р. за 1кг, (id=530af903-d932-4498-bb8d-9bf892bc8b49)
Покупка: 5кг - Арбуз (Продовольствие), цена 40р. за 1кг, (id=f4b15204-86dc-409a-8a55-f0d09bae777f)
Покупка: 2шт - Бумага А4 (Канцелярия), цена 300р. за 1шт, (id=63f1606a-cbaf-4980-b047-135ac9137762)
Покупка: 10шт - Мыло (Бытовая химия), цена 70р. за 1шт, (id=2a7a5f52-39a3-4644-b973-4b350294b66e)


In [ ]:
# Удаление покупки
del_purchase = purchases[1]['id']
r = requests.delete(f"http://127.0.0.1:8000/api/purchases/{del_purchase}")
del_purchase = r.json()
print('\nURI для отправки запроса:', r.request.url)
print(f"Удалена покупка: {del_purchase['quantity']}{del_purchase['unit']} {del_purchase['name']} ({del_purchase['group']}), цена {del_purchase['price']}р. за 1{del_purchase['unit']}")


URI для отправки запроса: http://127.0.0.1:8000/api/purchases/d426ce87-510e-4a3b-900c-f7dba8b63d61
Удалена покупка: 1шт Смартфон (Электроника), цена 12000р. за 1шт


In [ ]:
get_purchases()


Список покупок:
Покупка: 2кг - Помидоры (Продовольствие), цена 200р. за 1кг, (id=2ec65fa9-72bf-484e-8c05-a088c84fdf33)
Покупка: 5шт - Миф (Бытовая химия), цена 80р. за 1шт, (id=ccbfe4af-61f8-4fb3-bf92-9f6f3fb64213)
Покупка: 0.5кг - Сыр (Продовольствие), цена 800р. за 1кг, (id=530af903-d932-4498-bb8d-9bf892bc8b49)
Покупка: 5кг - Арбуз (Продовольствие), цена 40р. за 1кг, (id=f4b15204-86dc-409a-8a55-f0d09bae777f)
Покупка: 2шт - Бумага А4 (Канцелярия), цена 300р. за 1шт, (id=63f1606a-cbaf-4980-b047-135ac9137762)
Покупка: 10шт - Мыло (Бытовая химия), цена 70р. за 1шт, (id=2a7a5f52-39a3-4644-b973-4b350294b66e)


In [ ]:
r = requests.get('http://127.0.0.1:8000/api/purchases')
purchases = r.json()

# Получение покупки по идентификатору
purchase = purchases[1]['id']
r = requests.get(f"http://127.0.0.1:8000/api/purchases/{purchase}")
purchase = r.json()
print('\nURI для отправки запроса:', r.request.url)
print(f"Найдена покупка: {purchase['quantity']}{purchase['unit']} {purchase['name']} ({purchase['group']}), цена {purchase['price']}р. за 1{purchase['unit']}")


URI для отправки запроса: http://127.0.0.1:8000/api/purchases/ccbfe4af-61f8-4fb3-bf92-9f6f3fb64213
Найдена покупка: 5шт Миф (Бытовая химия), цена 80р. за 1шт


In [ ]:
# Получение суммы всех покупок и расходов по каждой группе
r = requests.get(f"http://127.0.0.1:8000/api/purchases_sum")
purchases_group_sum, purchases_sum = r.json()
print('Расходы по каждой группе')
for group in purchases_group_sum:
    print(group)

print('\nCуммa всех покупок')
print(purchases_sum)

Расходы по каждой группе
['Продовольствие', 1000.0]
['Бытовая химия', 1100]
['Канцелярия', 600]

Cуммa всех покупок
2700.0
